# Reinforcement Learning

In [2]:
import gym
import collections
from tensorboardX import SummaryWriter

In [3]:
ENV_NAME = "FrozenLake-v0"
GAMMA = 0.9
TEST_EPISODES = 20

In [4]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME) # create Frozen Lake
        self.state = self.env.reset() # Starting state.
        self.rewards = collections.defaultdict(float) # Reward Matrix
        self.transits = collections.defaultdict(collections.Counter) # trasition probs.
        self.values = collections.defaultdict(float) # state values. Vs

    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample() # Get a random action
            new_state, reward, is_done, _ = self.env.step(action) # Perform the action
            self.rewards[(self.state, action, new_state)] = reward # make an entry in reward table.
            self.transits[(self.state, action)][new_state] += 1 # transition count, later converted to probability.
            self.state = self.env.reset() if is_done else new_state # if episode done reset the environment else new state.

    def calc_action_value(self, state, action):
        target_counts = self.transits[(state, action)] # Get transition counts.
        total = sum(target_counts.values()) # Total transitions from state
        action_value = 0.0
        for tgt_state, count in target_counts.items(): # for each target state on doing action a instate s.
            reward = self.rewards[(state, action, tgt_state)] # Get reward.
            action_value += (count / total) * (reward + GAMMA * self.values[tgt_state]) # Action value for a state-action pair.
        return action_value

    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n): # for each action.
            action_value = self.calc_action_value(state, action) # For each state-action pair get action value Vs
            if best_value is None or best_value < action_value: # decide best action.
                best_value = action_value
                best_action = action
        return best_action

    def play_episode(self, env):
        total_reward = 0.0
        state = env.reset()
        while True:
            action = self.select_action(state) # Choose Best Action.
            new_state, reward, is_done, _ = env.step(action) # Do Action.
            self.rewards[(state, action, new_state)] = reward # Get reward.
            self.transits[(state, action)][new_state] += 1 # Make transition entry +1.
            total_reward += reward
            if is_done:
                break
            state = new_state
        return total_reward

    def value_iteration(self):
        for state in range(self.env.observation_space.n): # For every state.
            state_values = [self.calc_action_value(state, action) # Get state-action values.
                            for action in range(self.env.action_space.n)]
            self.values[state] = max(state_values) 

In [5]:
if __name__ == "__main__":
    test_env = gym.make(ENV_NAME)
    agent = Agent()
    writer = SummaryWriter(comment="-v-iteration")

    iter_no = 0
    best_reward = 0.0
    while True:
        iter_no += 1
        agent.play_n_random_steps(100)
        agent.value_iteration()

        reward = 0.0
        for _ in range(TEST_EPISODES):
            reward += agent.play_episode(test_env)
        reward /= TEST_EPISODES
        writer.add_scalar("reward", reward, iter_no)
        if reward > best_reward:
            print("Best reward updated %.3f -> %.3f" % (best_reward, reward))
            best_reward = reward
        if reward > 0.80:
            print("Solved in %d iterations!" % iter_no)
            break
    writer.close()

Best reward updated 0.000 -> 0.050
Best reward updated 0.050 -> 0.100
Best reward updated 0.100 -> 0.300
Best reward updated 0.300 -> 0.350
Best reward updated 0.350 -> 0.500
Best reward updated 0.500 -> 0.550
Best reward updated 0.550 -> 0.650
Best reward updated 0.650 -> 0.750
Best reward updated 0.750 -> 0.800
Best reward updated 0.800 -> 0.850
Solved in 88 iterations!
